In [ ]:

def dnn(batch_size=32,N_HIDDEN=2048,LN=4,drp=0.2):
    hard_targets = T.imatrix('hard_target')

    #network

    l_in = lasagne.layers.InputLayer(shape=(None, X_TRAIN.shape[1]))
    #
    l_hidden = lasagne.layers.DenseLayer(
        l_in,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d=lasagne.layers.dropout(l_hidden,drp)
    l_hidden2 = lasagne.layers.DenseLayer(
        d,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d2=lasagne.layers.dropout(l_hidden2,drp)
    l_hidden3 = lasagne.layers.DenseLayer(
        d2,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d3=lasagne.layers.dropout(l_hidden3,drp)
    l_hidden4 = lasagne.layers.DenseLayer(
        d3,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d4=lasagne.layers.dropout(l_hidden4,drp)
    l_hidden5 = lasagne.layers.DenseLayer(
        d4,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d5=lasagne.layers.dropout(l_hidden5,drp)
    l_hidden6 = lasagne.layers.DenseLayer(
        d5,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d6=lasagne.layers.dropout(l_hidden6,drp)
    if LN==3:
        go_in=d3
    elif LN==4:
        go_in=d4
    elif LN==5:
        go_in=d5
    elif LN==6:
        go_in=d6
    else:
        print 'wrong layers number'
    l_out = lasagne.layers.DenseLayer(
        go_in, num_units=NUM_STATES, nonlinearity=lasagne.nonlinearities.softmax)
    

    #soft for train and eval

    hard_loss_train = T.mean(lasagne.objectives.categorical_crossentropy(
        lasagne.layers.get_output(l_out, deterministic=False), hard_targets))
    hard_loss_eval = T.mean(lasagne.objectives.categorical_crossentropy(
        lasagne.layers.get_output(l_out, deterministic=True), hard_targets))
    
    loss_eval = hard_loss_eval

    loss_train = hard_loss_train

    all_params = lasagne.layers.get_all_params(l_out)


    #output of network
    get_train=theano.function([l_in.input_var],lasagne.layers.get_output(l_out, deterministic=True))
    #acc for hard
    get_acc = theano.function([l_in.input_var,hard_targets], T.mean(lasagne.objectives.categorical_accuracy(lasagne.layers.get_output(l_out, deterministic=True), hard_targets)))
    #get two losses
    get_loss = theano.function([l_in.input_var,hard_targets], hard_loss_eval)


    #load weights
    fname=PATH+('DnnWeight/tea'+'_%dL%d_drp%s_batch%d.npy' %(LN,N_HIDDEN,drp,batch_size))
    weights=np.load(fname)
    lasagne.layers.set_all_param_values(l_out, weights)
    
    #get soft
    SUB_RUN=50
    sub_size=X_TRAIN.shape[0]/SUB_RUN
    subs=list()
    for i in range(SUB_RUN):
        if i ==SUB_RUN-1:
            sub=get_train(X_TRAIN[i*sub_size:])
        else:
            sub=get_train(X_TRAIN[i*sub_size:sub_size*(i+1)])
        subs.append(sub)
    out=np.vstack(subs)
    out_vali=get_train(X_VALI)
    #get soft
    SUB_RUN=30
    sub_size=X_TEST.shape[0]/SUB_RUN
    subs=list()
    for i in range(SUB_RUN):
        if i ==SUB_RUN-1:
            sub=get_train(X_TEST[i*sub_size:])
        else:
            sub=get_train(X_TEST[i*sub_size:sub_size*(i+1)])
        subs.append(sub)
    out=np.vstack(subs)
    print "test acc:",get_acc(out,Y_TEST)
    print "test loss:",get_loss(out,Y_TEST)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_test",out)
    
    print "vali acc:",get_acc(out_vali,Y_VALI)
    print "vali loss:",get_loss(out_vali,Y_VALI)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_train",out)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_vali",out_vali)

dnn(batch_size=256,N_HIDDEN=3072,LN=5,drp=0.4)

In [ ]:

    out_vali=get_train(X_VALI)
    print "vali acc:",get_acc(out_vali,Y_VALI)
    print "vali loss:",get_loss(out_vali,Y_VALI)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_train",out)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_vali",out_vali)
    #get soft
    SUB_RUN=30
    sub_size=X_TEST.shape[0]/SUB_RUN
    subs=list()
    for i in range(SUB_RUN):
        if i ==SUB_RUN-1:
            sub=get_train(X_TEST[i*sub_size:])
        else:
            sub=get_train(X_TEST[i*sub_size:sub_size*(i+1)])
        subs.append(sub)
    out=np.vstack(subs)
    print "test acc:",get_acc(out,Y_TEST)
    print "test loss:",get_loss(out,Y_TEST)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_test",out)

In [1]:
import theano
import theano.tensor as T
import lasagne
import numpy as np
from keras.utils import np_utils
from keras.objectives import categorical_crossentropy


NUM_STATES=120
window_size=17
N_EPOCHS = 100

PATH="/home/jango/distillation/teacher/"
np.random.seed(55)

def MakeWindows(indata,window_size):
    outdata=[]
    for i in range(indata.shape[0]-window_size+1):
        outdata.append(np.hstack(indata[i:i+window_size]))
    return np.array(outdata)

def shufflelists(X_t,y_t):
    ri=np.random.permutation(len(X_t))
    X=X_t[ri]
    y=y_t[ri]
    return X,y

#-------------------------------------------------load data 
skip=window_size / 2
#train data
inputs =np.load(PATH+"DnnFile/tea_train.npy")
#make window
X_TRAIN=MakeWindows(inputs,window_size).astype('float32')

#load train targets
targets = np.load(PATH+"DnnFile/tea_train_target.npy")
Y_train=np_utils.to_categorical(targets,NUM_STATES)
Y_TRAIN=Y_train[skip:-skip].astype('int16')
#------------------------------------------------------------
#validation data
inputs =np.load(PATH+"DnnFile/tea_validation.npy")
#make window
X_VALI=MakeWindows(inputs,window_size).astype('float32')

#load train targets
targets = np.load(PATH+"DnnFile/tea_validation_target.npy")
Y_validation=np_utils.to_categorical(targets,NUM_STATES)
Y_VALI=Y_validation[skip:-skip].astype('int16')
#-------------------------------------------------load data 
skip=window_size / 2
#train data
inputs =np.load(PATH+"DnnFile/tea_test.npy")
#make window
X_TEST=MakeWindows(inputs,window_size).astype('float32')

#load train targets
targets = np.load(PATH+"DnnFile/tea_test_target.npy")
Y_train=np_utils.to_categorical(targets,NUM_STATES)
Y_TEST=Y_train[skip:-skip].astype('int16')

#***********************************************************************
print "train_set:"
print X_TRAIN.shape
print Y_TRAIN.shape
print "validation_set:"
print X_VALI.shape
print Y_VALI.shape

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5004)
/usr/local/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


train_set:
(913342, 1479)
(913342, 120)
validation_set:
(73143, 1479)
(73143, 120)


In [2]:
#-------------------------------------------------load data 
skip=window_size / 2
#train data
inputs =np.load(PATH+"DnnFile/tea_test.npy")
#make window
X_TEST=MakeWindows(inputs,window_size).astype('float32')

#load train targets
targets = np.load(PATH+"DnnFile/tea_test_target.npy")
Y_train=np_utils.to_categorical(targets,NUM_STATES)
Y_TEST=Y_train[skip:-skip].astype('int16')

In [5]:

def dnn(batch_size=32,N_HIDDEN=2048,LN=4,drp=0.2):
    #variable holders
    hard_targets = T.imatrix('hard_target')
    net_output = T.matrix('net_output')
    #network
    PATH="/home/jango/distillation/teacher/"
    l_in = lasagne.layers.InputLayer(shape=(None, X_TEST.shape[1]))
    #
    l_hidden = lasagne.layers.DenseLayer(
        l_in,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d=lasagne.layers.dropout(l_hidden,drp)
    l_hidden2 = lasagne.layers.DenseLayer(
        d,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d2=lasagne.layers.dropout(l_hidden2,drp)
    l_hidden3 = lasagne.layers.DenseLayer(
        d2,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d3=lasagne.layers.dropout(l_hidden3,drp)
    l_hidden4 = lasagne.layers.DenseLayer(
        d3,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d4=lasagne.layers.dropout(l_hidden4,drp)
    l_hidden5 = lasagne.layers.DenseLayer(
        d4,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d5=lasagne.layers.dropout(l_hidden5,drp)
    l_hidden6 = lasagne.layers.DenseLayer(
        d5,
        num_units=N_HIDDEN, W=lasagne.init.HeUniform(gain='relu'),b=lasagne.init.Constant(0.001),
        nonlinearity=lasagne.nonlinearities.rectify)
    d6=lasagne.layers.dropout(l_hidden6,drp)
    if LN==3:
        go_in=d3
    elif LN==4:
        go_in=d4
    elif LN==5:
        go_in=d5
    elif LN==6:
        go_in=d6
    else:
        print 'wrong layers number'
    l_out = lasagne.layers.DenseLayer(
        go_in, num_units=NUM_STATES, nonlinearity=lasagne.nonlinearities.linear)


    hard_loss_eval = T.mean(lasagne.objectives.categorical_crossentropy(T.nnet.softmax(net_output), hard_targets))
    
    all_params = lasagne.layers.get_all_params(l_out)

    #output of network
    get_train=theano.function([l_in.input_var],lasagne.layers.get_output(l_out, deterministic=True))
    #acc for hard
    get_acc = theano.function([net_output,hard_targets], T.mean(lasagne.objectives.categorical_accuracy(T.nnet.softmax(net_output), hard_targets)))
    #get two losses
    get_loss = theano.function([net_output,hard_targets], hard_loss_eval)

    #load weights
    fname=PATH+('DnnWeight%s/tea'+'_%dL%d_drp%s_batch%d.npy' %(SET,LN,N_HIDDEN,drp,batch_size))
    weights=np.load(fname)
    lasagne.layers.set_all_param_values(l_out, weights)
    
    #get soft
    SUB_RUN=10
    sub_size=X_TEST.shape[0]/SUB_RUN
    subs=list()
    for i in range(SUB_RUN):
        if i ==SUB_RUN-1:
            sub=get_train(X_TEST[i*sub_size:])
        else:
            sub=get_train(X_TEST[i*sub_size:sub_size*(i+1)])
        subs.append(sub)
    out=np.vstack(subs)
    print "test acc:",get_acc(out,Y_TEST)
    print "test loss:",get_loss(out,Y_TEST)
    #out_vali=get_train(X_VALI)
    #print "vali acc:",get_acc(out_vali,Y_VALI)
    #print "vali loss:",get_loss(out_vali,Y_VALI)
    #np.save(PATH[:-8]+"distrain/soft_targets/soft_train",out)
    #np.save(PATH[:-8]+"distrain/soft_targets/soft_vali",out_vali)
    np.save(PATH[:-8]+"distrain/soft_targets/soft_test",out)
dnn(batch_size=256,N_HIDDEN=3072,LN=5,drp=0.4)

test acc: 0.824670971684
test loss: 0.562096867405
